<a href="https://colab.research.google.com/github/marionwenger/DLColabNotebooks/blob/main/MRI_pretrained.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
# '/content/drive/MyDrive/DLforMRI'
# save files on drive or locally, they are deleted in colab after 12 hours at the latest

Mounted at /content/drive


In [2]:
!pip install split-folders

In [3]:
# copy from Nicola's resnet18-model notebook
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import copy # A module that provides functions for creating copies of objects, useful for avoiding unintended modifications to variables.
import os # A module that provides a way to interact with the operating system, allowing for tasks such as file and directory manipulation.
import torch
from PIL import Image # A module from the Python Imaging Library (PIL) that provides functionality for opening, manipulating, and saving various image file formats.
from torch.utils.data import Dataset # A PyTorch class that represents a dataset and provides an interface for accessing and processing the data during training.
import torchvision
import torchvision.transforms.v2 as transforms # A module from the torchvision library that provides common image transformations, such as resizing, cropping, and normalization.
# v2 is recommended and gets udpates
from torch.utils.data import random_split # A function from PyTorch that allows for randomly splitting a dataset into training and validation subsets.
from torch.optim.lr_scheduler import ReduceLROnPlateau # A PyTorch scheduler that adjusts the learning rate during training based on a specified metric, reducing it when the metric plateaus.
import torch.nn as nn # A module in PyTorch that provides classes for defining and building neural networks.
from torchvision import utils # A module from torchvision that contains utility functions for working with images, such as saving and visualizing them.
from torchvision.datasets import ImageFolder
import splitfolders
from torchsummary import summary
import torch.nn.functional as F
import pathlib


In [4]:
os.listdir(r'/content/drive/MyDrive/DLforMRI/Brain Tumor Data')

# ['metadata.csv', '.DS_Store', 'Healthy', 'Tumor']

['metadata.csv', '.DS_Store', 'Healthy', 'Tumor']

In [5]:
# Dataset Path
data_path_original = pathlib.Path(r'/content/drive/MyDrive/DLforMRI/Brain Tumor Data')

# Splitting dataset to train_set, val_set and test_set
splitfolders.ratio(data_path_original, output='MRI splitted', seed=1404, ratio=(0.6, 0.3, 0.1), move=True)
# set move=True if you want to move the files instead of copying.

# Copying files: 4610 files [10:50,  7.09 files/s]

Copying files: 4610 files [10:50,  7.09 files/s]


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/DLforMRI/Brain Tumor Data/MRI splitted'

In [6]:
# New dataset path
data_path_splitted = pathlib.Path(r'/content/MRI splitted')

os.listdir(r'/content/MRI splitted')

['test', 'train', 'val']

In [7]:
meta_data = pd.read_csv(r'/content/drive/MyDrive/DLforMRI/Brain Tumor Data/metadata.csv')
meta_data.head()


,Unnamed: 0,image,class,format,mode,shape
0,0,Cancer (1).jpg,tumor,JPEG,RGB,"(512, 512, 3)"
1,1,Cancer (1).png,tumor,PNG,L,"(300, 240)"
2,2,Cancer (1).tif,tumor,TIFF,RGB,"(256, 256, 3)"
3,3,Cancer (10).jpg,tumor,JPEG,RGB,"(512, 512, 3)"
4,4,Cancer (10).tif,tumor,TIFF,RGB,"(256, 256, 3)"


In [8]:
# define transformation
transform = transforms.Compose(
    [
        transforms.Resize((1000,1000)), # do not lose any pixels # Default is InterpolationMode.BILINEAR
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.RandomRotation(90),
        transforms.ToTensor() #,
        #transforms.Normalize(mean = [0.485, 0.456, 0.406],std = [0.229, 0.224, 0.225]) # Z-Transformation für die 3 Input-Channels
        # denke brauche keine Normalisation hier und falls doch, wäre es besser, die mean und std zu kennen...
        ]
)

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/v2/_deprecated.py:41: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.
  warnings.warn(


In [10]:
# Transform the train and validation data in their respective folders
train_set = torchvision.datasets.ImageFolder(data_path_splitted.joinpath("train"), transform=transform)
train_set.transform
val_set = torchvision.datasets.ImageFolder(data_path_splitted.joinpath("val"), transform=transform)
val_set.transform

# Compose(
#       Resize(size=[1000, 1000], interpolation=InterpolationMode.BILINEAR, antialias=True)
#       RandomHorizontalFlip(p=0.5)
#       RandomVerticalFlip(p=0.5)
#       RandomRotation(degrees=[-90.0, 90.0], interpolation=InterpolationMode.NEAREST, expand=False, fill=0)
#       ToTensor()
# )

Compose(
      Resize(size=[1000, 1000], interpolation=InterpolationMode.BILINEAR, antialias=True)
      RandomHorizontalFlip(p=0.5)
      RandomVerticalFlip(p=0.5)
      RandomRotation(degrees=[-90.0, 90.0], interpolation=InterpolationMode.NEAREST, expand=False, fill=0)
      ToTensor()
)

In [11]:
# brauch ich wahrscheinlich gar nicht... siehe keras weiter unten

# # import and load train, validation
# batch_size = 16
# # https://www.mdpi.com/2079-9292/12/4/964
# # The research study concluded that batch sizes between 2 and 32 produced good results and added that small batch sizes are more robust than high batch sizes.

# train_loader = torch.utils.data.DataLoader(train_set, batch_size = batch_size, shuffle = True, num_workers = 2)
# val_loader = torch.utils.data.DataLoader(val_set, batch_size = batch_size, shuffle = True, num_workers = 2)

# # print shape for Training data and Validation data
# for key, value in {'Training Data': train_loader, "Validation Data": val_loader}.items():
#     for X, Y in value:
#         print(f"{key}:")
#         print(f"Shape of X : {X.shape}")
#         print(f"Shape of Y : {Y.shape}")
#         print(f"Type  of Y : {Y.dtype}")
#         break

In [20]:
# print shape for Training data and Validation data
for key, value in {'Training Data': train_set, "Validation Data": val_set}.items():
    for X, Y in value:
        print(f"{key}:")
        print(f"Shape of X : {X.shape}")
        print(f"         Y : {Y      }") # Y is an int - should be 0 or 1 for 'Tumor' or 'Healthy' (folder names...)
        break

        # expected: torch.Size([1]) torch.int64 for Y
        # received: 'Y : 0'
        # TODO...???

Training Data:
Shape of X : torch.Size([3, 1000, 1000])
         Y : 0
Validation Data:
Shape of X : torch.Size([3, 1000, 1000])
         Y : 0


In [19]:
import tensorflow_hub as hub

efficientnet_model = hub.KerasLayer("https://www.kaggle.com/models/google/bit/frameworks/TensorFlow2/variations/m-r101x1/versions/1",
                                 trainable=False,
                                 input_shape=(3, 1000, 1000),
                                 name='efficientnet')

In [21]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import InputLayer, Dense, Conv2D, MaxPool2D, Flatten, Dropout
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

pretrained_model = Sequential([
    efficientnet_model,
    Dense(10, activation='softmax')
])

pretrained_model.compile(loss=CategoricalCrossentropy(),
                optimizer=Adam(),
                metrics=['accuracy'])

ValueError: Exception encountered when calling layer "efficientnet" (type KerasLayer).

in user code:

    File "/usr/local/lib/python3.10/dist-packages/tensorflow_hub/keras_layer.py", line 250, in call  *
        result = smart_cond.smart_cond(training,

    ValueError: Could not find matching concrete function to call loaded from the SavedModel. Got:
      Positional arguments (2 total):
        * <tf.Tensor 'x:0' shape=(None, 3, 1000, 1000) dtype=float32>
        * False
      Keyword arguments: {}
    
     Expected these arguments to match one of the following 4 option(s):
    
    Option 1:
      Positional arguments (2 total):
        * TensorSpec(shape=(None, None, None, 3), dtype=tf.float32, name='x')
        * True
      Keyword arguments: {}
    
    Option 2:
      Positional arguments (2 total):
        * TensorSpec(shape=(None, None, None, 3), dtype=tf.float32, name='input_1')
        * True
      Keyword arguments: {}
    
    Option 3:
      Positional arguments (2 tota

In [ ]:
X_train_tensor_rgb = tf.image.grayscale_to_rgb(X_train_tensor)
X_test_tensor_rgb = tf.image.grayscale_to_rgb(X_test_tensor)

history_5 = model_5.fit(x=X_train_tensor_rgb,
                        y=y_train_tensor,
                        steps_per_epoch=100,
                        epochs=15,
                        validation_data=(X_test_tensor_rgb, y_test_tensor),
                        callbacks=[scheduler],
                        verbose=0)

In [ ]:
best_lr_model_5 = get_best_learning_rate(history_5)
print(best_lr_model_5)

In [ ]:
odel_6 = clone_model(model_5)

model_6.compile(loss=CategoricalCrossentropy(),
                optimizer=Adam(best_lr_model_5),
                metrics=['accuracy'])

In [ ]:
tensorboard_callback_efficientnet_model = create_tensorboard_callbacks('efficientnet_model')

history_6 = model_6.fit(x=X_train_tensor_rgb,
                        y=y_train_tensor,
                        epochs=10,
                        validation_data=(X_test_tensor_rgb, y_test_tensor),
                        batch_size=32,
                        callbacks=[tensorboard_callback_efficientnet_model, early_stopping])

In [ ]:
model_6.summary()

In [ ]:
model_6.evaluate(x=X_test_tensor_rgb, y=y_test_tensor, steps=len(X_test_tensor))

In [ ]:
plot_result_model(history_6)